In [1]:
import numpy as np
import torch
torch.cuda.is_available()

True

In [2]:
!pip install datasets transformers

In [3]:
!pip install huggingface_hub

In [4]:
!apt-get install git-lfs # is an extension for Git that deals with large files more efficiently

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 18 not upgraded.


In [5]:
from datasets import load_dataset
imdb= load_dataset('imdb')

In [6]:
basic_train_dataset= imdb['train'].shuffle(seed=50).select([i for i in list(range(10000))])
basic_test_dataset= imdb['test'].shuffle(seed=50).select([i for i in list(range(10000))])
print(basic_train_dataset[0])
print(basic_test_dataset[0])

{'text': "The film lacks style, i mean original style. everything looks copied including action, first appearances in the movie, songs, dialog delivery, etc etc. Yes, there the goof-ups were original, like in the beginning a car is shown with number UP**** number and few seconds later it starts falling down a hill with number MH**** . That was one in many goof-ups of the movie :) Anything good in the movie? yes, for kareena fans, if there are any, Kareena in bikini. For akshay fans, his dialogs and action stunts. Thats it. nothing else. So watch it on your own risk and don't blame the director or actors. Director is already insane and actors, i pity them.", 'label': 0}
{'text': 'I wanted to like this movie. I really, really did. I was so excited when I saw the preview, which scared the hell out of me. But when I saw the actual film, I was disappointed. The acting is stilted, and the attempts at comedy are woefully out of place and forced. And I\'m sorry, but a boy being chased by a tur

In [7]:
from transformers import AutoTokenizer

In [8]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [9]:
def preprocess_function(examples):
  return tokenizer(examples['text'], truncation =True)

In [10]:
tokenized_train=basic_train_dataset.map(preprocess_function, batched=True)
tokenized_test=basic_test_dataset.map(preprocess_function, batched=True)

In [11]:
from transformers import DataCollatorWithPadding
'''it takes a list of examples and collate them into a format that can be fed into the model '''
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

***MODEL TRAINING***

In [12]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.weight', 'pre_classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
from datasets import load_metric

In [30]:
def compute_metrics(eval_pred):
  load_accuracy =load_metric('accuracy')
  load_f1 = load_metric('f1')
  logits, labels = eval_pred
  predictions = np.argmax(logits, axis=1)
  accuracy=load_accuracy.compute(predictions=predictions,references=labels)['accuracy']
  f1_score=load_f1.compute(predictions=predictions,references=labels)['f1']
  return {"accuracy": accuracy, "f1": f1}


In [31]:
'''from sklearn.metrics import f1_score

def compute_metrics(eval_pred):
    predictions, labels = eval_pred.predictions, eval_pred.label_ids
    accuracy = load_accuracy.compute(predictions=predictions, references=labels)['accuracy']
    f1_score_value = f1_score(labels, predictions, average='weighted')  # Change this line
    return {"accuracy": accuracy, "f1": f1_score_value}
'''

'from sklearn.metrics import f1_score\n\ndef compute_metrics(eval_pred):\n    predictions, labels = eval_pred.predictions, eval_pred.label_ids\n    accuracy = load_accuracy.compute(predictions=predictions, references=labels)[\'accuracy\']\n    f1_score_value = f1_score(labels, predictions, average=\'weighted\')  # Change this line\n    return {"accuracy": accuracy, "f1": f1_score_value}\n'

In [15]:
from huggingface_hub import notebook_login
notebook_login()

In [16]:
!pip install transformers[torch]

In [17]:
!pip install accelerate -U

In [18]:
from transformers import TrainingArguments

In [19]:
from transformers import TrainingArguments, Trainer
from torch.utils.data import DataLoader
repo_name = 'Sentiment_Analysis_using_BERT_With_10000_Samples'
#training_args= TrainingArguments
training_args = TrainingArguments(

    output_dir= repo_name,
    learning_rate = 2e-5,
    per_device_train_batch_size= 16,
    per_device_eval_batch_size =16,
    gradient_accumulation_steps= 2,
    num_train_epochs=5,
    weight_decay=0.01,
    save_strategy= 'epoch',
    push_to_hub=True

)

trainer =Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [20]:
trainer.train()

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
500,0.263000
1000,0.117400
1500,0.058000


TrainOutput(global_step=1560, training_loss=0.14198807584933745, metrics={'train_runtime': 2381.1933, 'train_samples_per_second': 20.998, 'train_steps_per_second': 0.655, 'total_flos': 6539580163543872.0, 'train_loss': 0.14198807584933745, 'epoch': 4.99})

In [22]:
trainer.push_to_hub()

'https://huggingface.co/Satyajithchary/Sentiment_Analysis_using_BERT_With_10000_Samples/tree/main/'

In [23]:
from transformers import pipeline
sentiment_model= pipeline(model='Satyajithchary/Sentiment_Analysis_using_BERT_With_10000_Samples')
sentiment_model(['This movie is awesome', 'This movie is disaster'])

[{'label': 'LABEL_1', 'score': 0.9927244782447815},
 {'label': 'LABEL_0', 'score': 0.9900264739990234}]